In [1]:
"""
Scratch notebook to build as we go. All functionality folded back into
relevant python files.
"""

'\nScratch notebook to build as we go. All functionality folded back into\nrelevant python files.\n'

In [6]:
# download, split, tokenize, and save the training and test data
!python3 ./data/prepare.py

Loading dataset shards: 100%|███████████████████| 80/80 [00:01<00:00, 53.16it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 7973700
    })
    val: Dataset({
        features: ['text'],
        num_rows: 40069
    })
})

writing /Users/vinay/src/gpt-2/./data/train.bin: 100%|█| 1024/1024 [01:58<00:00,
writing /Users/vinay/src/gpt-2/./data/val.bin: 100%|█| 1024/1024 [00:01<00:00, 7
